In [12]:
#%%capture
#!pip install rdkit
#!pip install torch_geometric
try:
  import rdkit
except:
  !pip install rdkit

try:
  import torch_geometric
except:
  !pip install torch_geometric
  

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
import seaborn as sns
import networkx as nx
from math import sqrt

from rdkit import Chem
from rdkit.Chem import AllChem, Draw, PandasTools, Descriptors

## **Loading Dataset**

In [14]:
df = pd.read_csv('lipophilicity_rdkit.csv')
df.shape

(4200, 213)

In [15]:
df.head()

,CMPD_CHEMBLID,exp,smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CHEMBL596271,3.54,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,8.838871,8.838871,0.008322,-4.082382,0.728444,38.208333,340.858,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL1951080,-1.18,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,15.117958,15.117958,0.920611,-6.843264,0.545587,34.121212,494.591,...,1,1,0,0,0,0,0,0,0,0
2,CHEMBL1771,3.69,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,13.321227,13.321227,0.250582,-3.906276,0.807761,36.619048,321.829,...,0,0,0,0,0,0,0,1,0,0
3,CHEMBL234951,3.37,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,14.213604,14.213604,0.187400,-4.272077,0.506650,33.321429,419.890,...,0,0,0,0,0,0,0,1,0,0
4,CHEMBL565079,3.10,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,14.167882,14.167882,0.671279,-4.810274,0.747686,30.607143,381.480,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_final = df[['CMPD_CHEMBLID', 'exp', 'smiles']]
df_final.to_csv('Lipophilicity_final_from_rdkit.csv', index=None)
df_final

,CMPD_CHEMBLID,exp,smiles
0,CHEMBL596271,3.54,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14
1,CHEMBL1951080,-1.18,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...
2,CHEMBL1771,3.69,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl
3,CHEMBL234951,3.37,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...
4,CHEMBL565079,3.10,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...
...,...,...,...
4195,CHEMBL496929,3.85,OCCc1ccc(NC(=O)c2cc3cc(Cl)ccc3[nH]2)cc1
4196,CHEMBL199147,3.21,CCN(C1CCN(CCC(c2ccc(F)cc2)c3ccc(F)cc3)CC1)C(=O...
4197,CHEMBL15932,2.10,COc1cccc2[nH]ncc12
4198,CHEMBL558748,2.65,Clc1ccc2ncccc2c1C(=O)NCC3CCCCC3


## **Data Analysis**

In [17]:
df_final.describe()

,exp
count,4200.000000
mean,2.186336
std,1.203004
min,-1.500000
25%,1.410000
50%,2.360000
75%,3.100000
max,4.500000


In [18]:
# make sure there are no duplicates
df['smiles'].nunique()

4200